## In this Notebook
* examples of applications of ADI 


In [2]:
using Random
using LinearAlgebra
using SparseArrays
using Plots

### Dynamical Systems
Given $x,u,y$ as state, input, and output vectors, we can work with the simple discrete-time linear system representing

$$\dot{x}=A x+B u, \quad y=C x$$

A banded system, determining vibrations across a guitar string? heat across a bar


In [ ]:
function 
    
    

### PDE

The example below is the 2D heat equation, or

$$u' = \alpha \nabla ^2 u$$

The boundary conditions are randomly generated in the function below.

In [17]:
n = 10
h = 1 / (n - 1)
A = (1 / h^2) * SymTridiagonal(2 * ones(n), -1 * ones(n-1));
A
tau = 0.2
x = repeat([collect(1:1:n)],n)
x = hcat(x...)
y = x'

C = log.(tau .+ abs.(x - y))




10×10 Array{Float64,2}:
 -1.60944    0.182322   0.788457  …   1.97408    2.10413    2.2192  
  0.182322  -1.60944    0.182322      1.82455    1.97408    2.10413 
  0.788457   0.182322  -1.60944       1.64866    1.82455    1.97408 
  1.16315    0.788457   0.182322      1.43508    1.64866    1.82455 
  1.43508    1.16315    0.788457      1.16315    1.43508    1.64866 
  1.64866    1.43508    1.16315   …   0.788457   1.16315    1.43508 
  1.82455    1.64866    1.43508       0.182322   0.788457   1.16315 
  1.97408    1.82455    1.64866      -1.60944    0.182322   0.788457
  2.10413    1.97408    1.82455       0.182322  -1.60944    0.182322
  2.2192     2.10413    1.97408       0.788457   0.182322  -1.60944 

In [ ]:
function heat_visualizer(A)
    
end

### Control Theory

### Image Denoising